### Simple CNN model with and without instrument inclusion 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, gc

from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score

import torch
import torchvision
import torch.nn as nn
from torch.utils.data.sampler import WeightedRandomSampler

from tqdm import tqdm

from src.dl_model_utils import get_transform_train,get_transform_valid, Cholec80Dataset, EarlyStopping, start_timer, end_timer_and_print

from src.models import ResNet18Model,ResNet18LSTM, ResNetBlock, ResNet50,ResNet50_ST,ResNet50_ST_Phase, ResNet50LSTM, Xception


In [2]:
INSTRUMENT_MAP = {
    '':0,
    'tool_Grasper':1,
    'tool_Grasper, tool_Hook':2,
    'tool_Hook':3,
    'tool_Grasper, tool_Irrigator':4,
    'tool_Irrigator':5,
    'tool_Bipolar':6,
    'tool_Grasper, tool_Bipolar':7,
    'tool_Grasper, tool_Clipper':8,
    'tool_Clipper':9,
    'tool_Grasper, tool_Scissors':10,
    'tool_SpecimenBag':11,
    'tool_Grasper, tool_SpecimenBag':12,
    'tool_Scissors':13,
    'tool_Grasper, tool_Bipolar, tool_Irrigator':14,
    'tool_Bipolar, tool_Irrigator':15,
    'tool_Bipolar, tool_SpecimenBag':16,
    'tool_Grasper, tool_Bipolar, tool_SpecimenBag':17,
    'tool_Irrigator, tool_SpecimenBag':18,
    'tool_Bipolar, tool_Irrigator, tool_SpecimenBag':19,
    'tool_Grasper, tool_Irrigator, tool_SpecimenBag':20,
    'tool_Grasper, tool_Clipper, tool_Irrigator':21,
    'tool_Clipper, tool_Irrigator':22,
    'tool_Grasper, tool_Clipper, tool_SpecimenBag':23,
    'tool_Grasper, tool_Scissors, tool_SpecimenBag':24,
    'tool_Hook, tool_Irrigator':25,
    'tool_Grasper, tool_Hook, tool_Irrigator':26,
    'tool_Bipolar, tool_Scissors':27
    }

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device Type : {}".format(device))


Device Type : cuda


In [4]:
#Import train and validation sets
train_df = pd.read_parquet('data/ordered_train_df.parquet')
val_df = pd.read_parquet('data/ordered_val_df.parquet')

#Sample train and val datasets at 1fps
train_df = train_df[train_df['frame'] % 25 == 0].reset_index(drop=True)
val_df = val_df[val_df['frame'] % 25 == 0].reset_index(drop=True)

train_df['tool_target'] = train_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].dot(train_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].columns + ', ').str.rstrip(', ').map(INSTRUMENT_MAP).values
val_df['tool_target'] = val_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].dot(val_df[['tool_Grasper', 'tool_Bipolar',
        'tool_Hook', 'tool_Scissors', 'tool_Clipper', 'tool_Irrigator',
        'tool_SpecimenBag']].columns + ', ').str.rstrip(', ').map(INSTRUMENT_MAP).values

### Training Cycle

In [6]:
def model_train_validate_cnn(train_df, val_df,epochs=100, model_num=0, optimizer_num = 0,batch_size = 128, learning_rate = 1e-3, weight_decay=5e-4,momentum=0.9,patience=20, patience_delta = 0.0005):
    '''
    Model nums:

    Model 0: ResNet18
    Model 1: Resnet50 - Pre-Trained
    Model 2: ResNet18 w/LSTM
    Model 3: ResNet50 - Pre-Trained w/LSTM
    Model 4: Xception - Pre-Trained
    Model 5: Resnet50 - OD Phase

    '''

    #Get train label weights and convert to torch tensor
    phase_weights=class_weight.compute_class_weight('balanced',classes=np.unique(train_df['phase']),y=train_df['phase'].to_numpy())
    phase_weights=torch.tensor(phase_weights,dtype=torch.float)

    #Create datasets for phase segmentation
    trainset = Cholec80Dataset(train_df,  get_transform_train())
    validationset = Cholec80Dataset(val_df,  get_transform_valid())

    #Shuffle for non-lstm model. For lstm we preserve the sequence
    if model_num == 2 or model_num == 3:
        trainloader = torch.utils.data.DataLoader(
            trainset, batch_size=batch_size, shuffle=False
        )

    else:
        trainloader = torch.utils.data.DataLoader(
            trainset, batch_size=batch_size, shuffle=True
        )


    validationloader = torch.utils.data.DataLoader(
        validationset, batch_size=batch_size, shuffle=False
    )

    #Initialise two required loss functions
    criterion = nn.CrossEntropyLoss(weight=phase_weights)
    criterion_tool = nn.BCEWithLogitsLoss(reduction='mean')
    if device == 'cuda':
        criterion.to(device)
  

    #Initialize early stopping
    early_stopping = EarlyStopping(patience, patience_delta)


    #Model selection
    if model_num == 0:
        model = ResNet18Model(ResNetBlock)
        model_name = 'resnet_18_ord'
        
    elif model_num == 1:
        model = ResNet50()
        model.freeze()
        model_name = 'resnet_50_ord'
    elif model_num == 2:
        model = ResNet18LSTM()
        model_name = 'resnet_18_lstm_ord'
    elif model_num == 3:
        model = ResNet50LSTM()
        model_name = 'resnet_50_lstm_ord'
    elif model_num == 4:
        model = Xception()
        model.freeze()
        model_name = 'xception_ord'
    elif model_num == 5:
        model = ResNet50_ST_Phase()
        model.freeze()
        model_name = 'resnet_50_od_phase_ord'


    #Select optimizer for job
    optimizer = None
    if optimizer_num == 0:
        optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, momentum=momentum,weight_decay=weight_decay)
    elif optimizer_num == 1:
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)

    if device == 'cuda':
        model.to(device)

    start_timer()
    for epoch in range(0,epochs):
        print(f'Training Epoch: {epoch+1} of {epochs}..')

        train_loss = 0.0
        train_correct = 0
        total = 0.0

        # Outputs for HMM
        img_path = []
        true_labels = []
        soft_max_out = []
        predicted_labels = []

        model.train()
        for i, (train_inputs,train_img_id, train_labels,train_tools,train_tool_targets) in tqdm(enumerate(trainloader, 0)):
            img_path = img_path + list(train_img_id)
            true_labels = true_labels + train_labels.tolist()
            if device == 'cuda':
                train_inputs, train_labels, train_tools = train_inputs.to(device), train_labels.to(device), train_tools.to(device)

            optimizer.zero_grad()
            # if model_num < 2:
            if model_num != 5:
                train_outputs  = model.forward(train_inputs)
            else:
                train_outputs, _ = model.forward(train_inputs)
            
            _, train_preds = torch.max(train_outputs.data, 1)
            loss = criterion(train_outputs, train_labels)

            loss.backward()
            optimizer.step()

                
            # else:

                  
                
                
            #     tool_outputs, train_outputs  = model.forward(train_inputs)
            #     tool_outputs = tool_outputs.data

                
                

            #     train_tool_outputs = (tool_outputs > 0.5).float()

                
            #     train_tool_outputs = train_tool_outputs.float()

            #     tool_l = criterion_tool(train_tool_outputs, train_tools.float())

            #     _, train_preds = torch.max(train_outputs.data, 1)

            #     phase_l = criterion(train_outputs, train_labels)

            #     loss = tool_l + phase_l
            #     loss.backward()
            #     optimizer.step()

            soft_max_out = soft_max_out + train_outputs.cpu().detach().tolist()
            predicted_labels = predicted_labels + train_preds.cpu().tolist()

            train_loss += loss.item()
            total += train_labels.size(0)
            train_correct += train_preds.eq(train_labels).sum().item()

            
        hmm_df = pd.DataFrame(
            {'img_path': img_path,
                'true_labels': true_labels,
                'cnn_output': soft_max_out,
                'predicted_labels': predicted_labels
                })

        
        print(len(trainloader), 'Training Loss Phase: %.5f | Training Acc Phase: %.5f%% (%d/%d)'
        % (train_loss / len(trainloader), 100. * train_correct / total, train_correct, total))

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0.0
            
        with torch.no_grad():
            for j, (val_inputs,val_img_id, val_labels,val_tools,val_tool_targets) in enumerate(validationloader):
                if device == 'cuda':
                    val_inputs, val_labels, val_tools = val_inputs.to(device), val_labels.to(device), val_tools.to(device)

                if model_num != 5:
                    val_outputs = model.forward(val_inputs)
                else:
                    val_outputs, _ = model.forward(val_inputs)
    
                _, val_preds = torch.max(val_outputs.data, 1)
                loss = criterion(val_outputs, val_labels)

                # else:
                #     tool_outputs, val_outputs = model.forward(val_inputs)
                #     tool_outputs = tool_outputs.data


                #     val_tool_outputs = (tool_outputs > 0.5).float()

                #     tool_l = criterion_tool(val_tool_outputs, val_tools.float())
        
                #     _, val_preds = torch.max(val_outputs.data, 1)
                #     phase_l = criterion(val_outputs, val_labels)
                #     loss = tool_l + phase_l


                val_loss += loss.item()
                val_total += val_labels.size(0)
                val_correct += val_preds.eq(val_labels).sum().item()

        
            print(len(validationloader), 'Validation Loss Phase: %.5f | Validation Acc Phase: %.5f%% (%d/%d)'
                % (val_loss / len(validationloader), 100. * val_correct / val_total, val_correct, val_total))
    
            scheduler.step(val_loss)
            #Get average for early stopping check and call the early stopping class to perform calc
            val_loss_avg = val_loss/len(validationloader)
            early_stopping(val_loss_avg, model, model_name, hmm_df)

            #If limit reached stop execution
            if early_stopping.stop:
                break
              
    end_timer_and_print()
    
  
            

In [7]:
start_time = None
model_train_validate_cnn(train_df, val_df,batch_size=256, model_num=3)

c:\Users\vkula\anaconda3\lib\site-packages\torch\cuda\memory.py:278: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


Training Epoch: 1 of 100..


338it [06:17,  1.12s/it]


338 Training Loss Phase: 1.75155 | Training Acc Phase: 37.47553% (32356/86339)
84 Validation Loss Phase: 1.71584 | Validation Acc Phase: 35.89648% (7698/21445)
Training Epoch: 2 of 100..


338it [05:53,  1.05s/it]


...

338 Training Loss Phase: 0.04405 | Training Acc Phase: 98.12483% (84720/86339)
84 Validation Loss Phase: 1.14157 | Validation Acc Phase: 71.19142% (15267/21445)
Early Stopping count is at: 5 maximum is: 20
Training Epoch: 26 of 100..


338it [05:28,  1.03it/s]


338 Training Loss Phase: 0.03302 | Training Acc Phase: 98.67036% (85191/86339)
84 Validation Loss Phase: 1.02560 | Validation Acc Phase: 74.59548% (15997/21445)
Early Stopping count is at: 6 maximum is: 20
Training Epoch: 27 of 100..


131it [02:10,  1.01it/s]


KeyboardInterrupt: 

In [6]:

def train_validate_tool_detect(train_df, val_df, method=0, device='cuda',epochs=100, optimizer_num = 0,batch_size = 128, learning_rate = 1e-3, weight_decay=5e-4,momentum=0.9,patience=20, patience_delta = 0.0005):
    '''
    Model nums:
    CNN tool detection
    Model 0: ResNet50

    '''
    
    #Get train label weights and convert to torch tensor
    tool_weights=class_weight.compute_class_weight('balanced',classes=np.unique(train_df['tool_target']),y=train_df['tool_target'].to_numpy())
    tool_weights=torch.tensor(tool_weights,dtype=torch.float)
    print(f'Weighted classes: {tool_weights}')

    #Create datasets for phase segmentation
    trainset = Cholec80Dataset(train_df,  get_transform_train())
    validationset = Cholec80Dataset(val_df,  get_transform_valid())

    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True
    )
    validationloader = torch.utils.data.DataLoader(
        validationset, 
        batch_size=batch_size, 
        shuffle=False
    )

    #Initialise tool detection loss
    criterion = nn.BCEWithLogitsLoss(reduction='sum')
    criterion_tool = nn.CrossEntropyLoss(weight=tool_weights)
    if device == 'cuda':
        criterion.to(device)
        criterion_tool.to(device)
        
    #Initialize early stopping
    early_stopping = EarlyStopping(patience, patience_delta)


    #Model selection
    model = ResNet50_ST()
    model_name = 'ResNet50_od'

    # elif model_num == 2:
    #     model = DeiT_OD_phase()
    #     model_name = 'DeiT3_ord_od'
    #     model.freeze()

    #Final calculated learning rate
    learning_rate_final = learning_rate * batch_size / 512

    #Select optimizer for job
    optimizer = None
    if optimizer_num == 0:
        optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate_final, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_num == 1:
        # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
        optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate_final, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True)


    if device == 'cuda':
        model.to(device)

    start_timer()
    for epoch in range(0,epochs):
        print(f'Training Epoch: {epoch+1} of {epochs}..')

        train_loss = 0.0
        train_correct = 0
        total = 0.0
        batch_gt = []
        batch_pred = []

        # Outputs for HMM
        img_path = []
        true_labels = []
        softmax_out = []
        predicted_labels = []


        model.train()
        for i, (train_inputs,train_img_id, train_labels,train_tools,train_tool_targets) in tqdm(enumerate(trainloader, 0)):
            img_path = img_path + list(train_img_id)
            true_labels = true_labels + train_tool_targets.tolist()

            if device == 'cuda':
                train_inputs, train_labels, train_tools,train_tool_targets = train_inputs.to(device), train_labels.to(device), train_tools.to(device), train_tool_targets.to(device)

            optimizer.zero_grad()

            train_outputs, sigmoid_outputs  = model.forward(train_inputs)
            if method ==0:
                train_preds = (sigmoid_outputs > 0.5).float()
            else:
                _, train_preds = torch.max(train_outputs.data, 1)

            if method == 0:
                loss = criterion(train_outputs, train_tools)
            else:
                loss = criterion_tool(train_outputs, train_tool_targets)
                
            loss.backward()
            optimizer.step()

            softmax_out = softmax_out + train_outputs.cpu().detach().tolist()
            predicted_labels = predicted_labels + train_preds.cpu().tolist()

            train_loss += loss.item()
            if method == 0:
                batch_gt.append(np.concatenate(train_tools.cpu().tolist()))
                batch_pred.append(np.concatenate(train_preds.cpu().tolist()))
            else:
                total += train_labels.size(0)
                train_correct += train_preds.eq(train_tool_targets).sum().item()
                
        if method == 0:
            all_batch_gt = np.concatenate(batch_gt)
            all_batch_pred = np.concatenate(batch_pred)

        hmm_df = pd.DataFrame(
            {'img_path': img_path,
                'true_labels': true_labels,
                'cnn_output': softmax_out,
                'predicted_labels': predicted_labels
                })
        if method == 0:
            print(len(trainloader), 'Training Loss Phase: %.5f | Training Acc Phase: %.5f%%'
            % (train_loss / len(trainloader),100. * accuracy_score(all_batch_gt,all_batch_pred)))
        else:
            print(len(trainloader), 'Training Loss Phase: %.5f | Training Acc Phase: %.5f%% (%d/%d)'
            % (train_loss / len(trainloader), 100. * train_correct / total, train_correct, total))

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0.0
        val_batch_gt = []
        val_batch_pred = []
            
        with torch.no_grad():
            for j, (val_inputs,val_img_id, val_labels,val_tools,val_tool_targets) in enumerate(validationloader):
                if device == 'cuda':
                    val_inputs, val_labels, val_tools, val_tool_targets = val_inputs.to(device), val_labels.to(device), val_tools.to(device), val_tool_targets.to(device)

                val_outputs, val_sigmoid_outputs = model.forward(val_inputs)
                if method == 0:
                    val_preds = (val_sigmoid_outputs > 0.5).float()
                else:
                    _, val_preds = torch.max(val_outputs.data, 1)

                if method == 0:
                    loss = criterion(val_outputs, val_tools)
                else:
                    loss = criterion_tool(val_outputs, val_tool_targets)

                val_loss += loss.item()
                if method == 0:
                    val_batch_gt.append(np.concatenate(val_tools.cpu().tolist()))
                    val_batch_pred.append(np.concatenate(val_preds.cpu().tolist()))
                else:
                    val_total += val_labels.size(0)
                    val_correct += val_preds.eq(val_tool_targets).sum().item()

            if method == 0:
                all_batch_gt = np.concatenate(val_batch_gt)
                all_batch_pred = np.concatenate(val_batch_pred)

            if method == 0:
                print(len(validationloader), 'Validation Loss Phase: %.5f | Validation Acc Phase: %.5f%%'
                % (val_loss / len(validationloader), 100. * accuracy_score(all_batch_gt,all_batch_pred)))
            else:
                print(len(validationloader), 'Validation Loss Phase: %.5f | Validation Acc Phase: %.5f%% (%d/%d)'
                % (val_loss / len(validationloader), 100. * val_correct / val_total, val_correct, val_total))
    
            scheduler.step(val_loss)
            #Get average for early stopping check and call the early stopping class to perform calc
            val_loss_avg = val_loss/len(validationloader)
            early_stopping(val_loss_avg, model, model_name, hmm_df)

            #If limit reached stop execution
            if early_stopping.stop:
                break

    end_timer_and_print()

In [7]:

start_time = None
train_validate_tool_detect(train_df, val_df, batch_size=256)


Weighted classes: tensor([3.5000e-01, 2.6292e-01, 9.2914e-02, 2.0479e-01, 1.4761e+00, 2.3115e+00,
        2.8420e+00, 1.9868e+00, 1.4197e+00, 2.9995e+00, 3.1497e+00, 4.3800e+00,
        8.7081e-01, 5.6997e+00, 6.1303e+00, 1.3465e+01, 2.4280e+01, 9.0960e+00,
        1.0560e+01, 1.5813e+01, 5.7852e+00, 2.5696e+02, 7.7088e+02, 3.0835e+03,
        1.1013e+02, 5.4097e+01, 2.2673e+01, 4.0573e+01])


c:\Users\vkula\anaconda3\lib\site-packages\torch\cuda\memory.py:278: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


Training Epoch: 1 of 100..


338it [04:54,  1.15it/s]


338 Training Loss Phase: 146.00623 | Training Acc Phase: 96.94510%
84 Validation Loss Phase: 264.77233 | Validation Acc Phase: 95.32025%
Training Epoch: 2 of 100..


...

338 Training Loss Phase: 0.55745 | Training Acc Phase: 99.99437%
84 Validation Loss Phase: 404.60815 | Validation Acc Phase: 97.12953%
Early Stopping count is at: 25 maximum is: 20
Training Epoch: 29 of 100..


338it [04:49,  1.17it/s]


338 Training Loss Phase: 0.55790 | Training Acc Phase: 99.99487%
84 Validation Loss Phase: 401.40203 | Validation Acc Phase: 97.18882%
Early Stopping count is at: 26 maximum is: 20
Training Epoch: 30 of 100..


35it [00:33,  1.03it/s]


KeyboardInterrupt: 

In [6]:
# running_loss_history = []
# running_corrects_history = []
# val_running_loss_history = []
# val_running_corrects_history = []

# start_timer()
# # training epochs
# for epoch in range(0, EPOCHS):

#     correct = 0.0
#     total = 0.0
#     running_loss = 0.0

#     # Outputs for HMM
#     img_path = []
#     true_labels = []
#     soft_max_out = []
#     predicted_labels = []

#     for i, data in tqdm(enumerate(trainloader, 0)):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs,img_id, labels,tools  = data
#         img_path = img_path + list(img_id)


#         true_labels = true_labels + labels.tolist()
        
#         inputs, labels = inputs.to(device), labels.to(device)
#         inputs_tools = tools.to(device)


#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         if USE_TOOL:
#             outputs = model(inputs, inputs_tools)
#         else:
#             outputs = model(inputs)

#         soft_max_out = soft_max_out + outputs.cpu().detach().tolist()


#         loss = criterion(outputs, labels)

#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         _, predicted = outputs.max(1)
#         predicted_labels = predicted_labels + predicted.cpu().tolist()

#         total += labels.size(0)
#         correct += predicted.eq(labels).sum().item()

#     hmm_df = pd.DataFrame(
#         {'img_path': img_path,
#             'true_labels': true_labels,
#             'cnn_output': soft_max_out,
#             'predicted_labels': predicted_labels
#             })

#     print(len(trainloader), 'Training Loss: %.5f | Training Acc: %.5f%% (%d/%d)'
#             % (running_loss / len(trainloader), 100. * correct / total, correct, total))

#     model.eval()
#     val_loss = 0.0
#     val_correct = 0.0
#     val_total = 0.0

#     with torch.no_grad():
#         for j, (val_inputs,val_img_id, val_labels,val_tools) in enumerate(validationloader):
#             val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
#             val_inputs_tools = val_tools.to(device)


            

#             if USE_TOOL:
#                 val_outputs = model(val_inputs, val_inputs_tools)
#             else:
#                 val_outputs = model(val_inputs)

#             validation_loss = criterion(val_outputs, val_labels)

#             val_loss += validation_loss.item()
#             _, val_predicted = val_outputs.max(1)
#             val_total += val_labels.size(0)
#             val_correct += val_predicted.eq(val_labels).sum().item()

#         print(len(validationloader), 'Validation Loss: %.5f | Validation Acc: %.5f%% (%d/%d)'
#                 % (val_loss / len(validationloader), 100. * val_correct / val_total, val_correct, val_total))
#         scheduler.step(val_loss)
        
#         #Get average for early stopping check and call the early stopping class to perform calc
#         val_loss_avg = val_loss/len(validationloader)
#         early_stopping(val_loss_avg, model, MODEL_NAME)

#         #If limit reached stop execution
#         if early_stopping.stop:
#             break

#     epoch_loss = running_loss / len(trainloader)  # loss per epoch
#     epoch_acc = correct / total  # accuracy per epoch
#     running_loss_history.append(epoch_loss)  # appending for displaying
#     running_corrects_history.append(epoch_acc)

#     val_epoch_loss = val_loss / len(validationloader)
#     val_epoch_acc = val_correct / val_total
#     val_running_loss_history.append(val_epoch_loss)
#     val_running_corrects_history.append(val_epoch_acc)


# plt.style.use('ggplot')
# plt.plot(running_loss_history, label='training loss')
# plt.plot(val_running_loss_history, label='validation loss')
# plt.title("Training validation loss curve")
# plt.legend()
# print("Finished Training")
# end_timer_and_print()